# **1. 서울 열린데이터 광장**

[서울 열린데이터 광장](https://data.seoul.go.kr/)(Seoul Open Data Plaza)은 서울시에서 운영하는 공공데이터 개방 플랫폼입니다. 시민, 연구자, 기업 등이 서울시에서 생성한 다양한 공공데이터를 자유롭게 활용할 수 있도록 제공하고 있습니다. 이를 통해 데이터 기반의 창의적인 아이디어와 혁신을 촉진하며, 시민들의 정보 접근성을 높이고 공공서비스를 개선하는 데 기여하고 있습니다.

# **2. JSON**
JSON(JavaScript Object Notation)은 자바스크립트 객체 표기법을 기반으로 한 데이터 교환 형식으로, 사람도 읽기 쉽고 기계도 쉽게 분석하고 생성할 수 있는 텍스트 기반의 데이터 포맷입니다. 일반적으로 서버와 클라이언트 간에 데이터를 주고받을 때 많이 사용되며, 구조는 키-값 쌍으로 이루어진 객체 형태나 배열 형태를 사용합니다. 언어에 독립적이면서도 자바스크립트와 호환성이 뛰어나 웹 개발뿐만 아니라 다양한 API 통신, 데이터 저장 및 전송 등에 널리 활용됩니다.

### JSON 문법 구조

1. 객체(Object)
* 중괄호 {} 사용
* 내부에 "key": value 형식으로 속성-값 쌍을 나열
* 각 쌍은 쉼표 ,로 구분
* 값으로는 문자열, 숫자, 불리언, 배열, 객체 모두 가능
{
  "name": "김사과",
  "age": 20,
  "isStudent": true,
  "skills": ["HTML", "CSS", "JavaScript"]
}

```
{
  "name": "김사과",
  "age": 20,
  "isStudent": true,
  "skills": ["HTML", "CSS", "JavaScript"]
}
```

2. 배열(Array)
* 대괄호 [] 사용
* 여러 개의 값을 순서대로 나열
* 요소는 숫자, 문자열, 객체 등 다양한 데이터 가능
```
[
  "사과",
  "바나나",
  "오렌지"
]
```

3. JSON 안에 JSON (중첩 구조)
* 객체 안에 객체, 객체 안에 배열도 가능
* 실제 API 응답이나 설정 파일에서 많이 사용됨
```
{
  "name": "김사과",
  "address": {
    "city": "서울",
    "zipcode": "12345"
  },
  "skills": ["HTML", "CSS", "JavaScript"]
}
```

4. JSON 규칙 (중요)
* 문자열은 반드시 큰따옴표 " ", 작은따옴표 ' '는 에러 발생
* 키(key)도 반드시 쌍따옴표 " "로 감싸야 함
* 값(value)에는 객체, 배열, 문자열, 숫자, 불리언, null 사용 가능
* 마지막 항목 뒤에는 쉼표 금지
* JSON은 순수 데이터 형식이기 때문에 //, /* */ 같은 주석 사용 불가

# **3. 서울시 공공자전거 실시간 대여정보**

In [1]:
import requests
import folium
import json
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [3]:
base_url = 'http://openapi.seoul.go.kr:8088/7352417744616f67383255444f6179/json/bikeList/1/5/'
response = requests.get(base_url)
# print(request)
json_data = response.json()
json_data

{'rentBikeStatus': {'list_total_count': 5,
  'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다.'},
  'row': [{'rackTotCnt': '15',
    'stationName': '102. 망원역 1번출구 앞',
    'parkingBikeTotCnt': '9',
    'shared': '60',
    'stationLatitude': '37.55564880',
    'stationLongitude': '126.91062927',
    'stationId': 'ST-4'},
   {'rackTotCnt': '14',
    'stationName': '103. 망원역 2번출구 앞',
    'parkingBikeTotCnt': '19',
    'shared': '136',
    'stationLatitude': '37.55495071',
    'stationLongitude': '126.91083527',
    'stationId': 'ST-5'},
   {'rackTotCnt': '13',
    'stationName': '104. 합정역 1번출구 앞',
    'parkingBikeTotCnt': '21',
    'shared': '162',
    'stationLatitude': '37.55073929',
    'stationLongitude': '126.91508484',
    'stationId': 'ST-6'},
   {'rackTotCnt': '5',
    'stationName': '105. 합정역 5번출구 앞',
    'parkingBikeTotCnt': '15',
    'shared': '300',
    'stationLatitude': '37.55000687',
    'stationLongitude': '126.91482544',
    'stationId': 'ST-7'},
   {'rackTotCnt': '12

In [4]:
json_data.get("rentBikeStatus", {}).get("RESULT", {}).get("CODE", "")

'INFO-000'

In [5]:
json_data["rentBikeStatus"]["RESULT"]["CODE"]

'INFO-000'

In [8]:
def fetch_bike_data():
    base_url = "http://openapi.seoul.go.kr:8088/7352417744616f67383255444f6179/json/bikeList/"
    start = 1
    end = 1000
    step = 1000
    data_frames = []

    while True:
        url = f"{base_url}{start}/{end}/"
        response = requests.get(url)

        if response.status_code != 200:
            print(f"Status Code: {response.status_code}")
            break

        json_data = response.json()

        try:
            rent_bike_status = json_data["rentBikeStatus"]
            result_code = rent_bike_status["RESULT"]["CODE"]
        except KeyError:
            print("JSON 오류")
            break

        if result_code == "INFO-200":
            print("데이터 없음")
            break
        elif result_code == "INFO-000":
            print(f"시작: {start} 끝: {end}.")
            try:
                bike_data = rent_bike_status["row"]
                if bike_data:
                    df = pd.DataFrame(bike_data)
                    data_frames.append(df)
            except KeyError:
                print("데이터를 찾을 수 없음")
        else:
            print(f"result code: {result_code}")
            break

        start += step
        end += step

    if data_frames:
        final_df = pd.concat(data_frames, ignore_index=True)
        return final_df
    else:
        return pd.DataFrame()

In [10]:
bike_data_df = fetch_bike_data()

시작: 1 끝: 1000.
시작: 1001 끝: 2000.
시작: 2001 끝: 3000.
JSON 오류


In [12]:
bike_data_df

,rackTotCnt,stationName,parkingBikeTotCnt,shared,stationLatitude,stationLongitude,stationId
0,15,102. 망원역 1번출구 앞,10,67,37.55564880,126.91062927,ST-4
1,14,103. 망원역 2번출구 앞,17,121,37.55495071,126.91083527,ST-5
2,13,104. 합정역 1번출구 앞,22,169,37.55073929,126.91508484,ST-6
3,5,105. 합정역 5번출구 앞,16,320,37.55000687,126.91482544,ST-7
4,12,106. 합정역 7번출구 앞,7,58,37.54864502,126.91282654,ST-8
...,...,...,...,...,...,...,...
2757,8,6185.가양나들목,6,75,37.57341003,126.84345245,ST-3418
2758,12,6187.마곡119안전센터 맞은편,5,42,37.55534744,126.82072449,ST-3415
2759,10,6188.금호아파트,19,190,37.55619049,126.86463928,ST-3419
2760,11,6189.데시앙플렉스 지식산업센터,6,55,37.56448364,126.84830475,ST-3424


In [13]:
bike_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2762 entries, 0 to 2761
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   rackTotCnt         2762 non-null   object
 1   stationName        2762 non-null   object
 2   parkingBikeTotCnt  2762 non-null   object
 3   shared             2762 non-null   object
 4   stationLatitude    2762 non-null   object
 5   stationLongitude   2762 non-null   object
 6   stationId          2762 non-null   object
dtypes: object(7)
memory usage: 151.2+ KB


In [14]:
'''
rackTotCnt	거치대개수
parkingBikeTotCnt	자전거주차총건수
shared	거치율
stationLatitude	위도
stationLongitude	경도
stationId	대여소ID
stationName	대여소이름
'''
bike_data_df.columns

Index(['rackTotCnt', 'stationName', 'parkingBikeTotCnt', 'shared',
       'stationLatitude', 'stationLongitude', 'stationId'],
      dtype='object')

In [15]:
bike_data_df.shape

(2762, 7)

In [16]:
bike_data_df['stationLongitude'] = bike_data_df['stationLongitude'].astype(float)
bike_data_df['stationLatitude'] = bike_data_df['stationLatitude'].astype(float)

In [17]:
bike_map = folium.Map(location=[bike_data_df['stationLatitude'].mean(),
                                bike_data_df['stationLongitude'].mean()],
                                zoom_start=12)

for index, data in bike_data_df.iterrows():
    popup_str = '{} 자전거주차총건수:{}대'.format(
        data['stationName'], data['parkingBikeTotCnt']
    )
    popup = folium.Popup(popup_str, max_width=600)
    folium.Marker(location=[data['stationLatitude'], data['stationLongitude']],
                  popup=popup).add_to(bike_map)

bike_map